# PICRUSt tutorial



A

Set path to analysis directory.

In [ ]:
ANALYSIS_PATH=/srv/home/anavarro/hidro_agave_diversidad
echo ANALYSIS_PATH

TMPDIR=/srv/home/anavarro/tmp

14) cluster-features with closed_reference.

In [ ]:
{
echo '#!/bin/bash'
printf 'export TMPDIR=%s\n' "$TMPDIR"

echo 'qiime vsearch cluster-features-closed-reference \'
printf -- '--i-sequences %s/2_resultados/4_deblur_output/representative_sequences.qza \\\n' "$ANALYSIS_PATH"
printf -- '--i-table %s/2_resultados/4_deblur_output/table.qza \\\n' "$ANALYSIS_PATH"
printf -- '--i-reference-sequences %s/7_databases/gg_13_8_otus97.qza \\\n' "$ANALYSIS_PATH"
echo '--p-perc-identity 0.97 \'
echo '--p-threads 0 \'
printf -- '--output-dir %s/2_resultados/picrust/14_closedRef_forPICRUSt_13_8 \\\n' "$ANALYSIS_PATH"
} > 14_picrust_clustering.sh

In [ ]:
{
printf 'executable = %s/3_ejecutables/picrust_execs/14_picrust_clustering.sh\n' "$ANALYSIS_PATH"
echo 'getenv= True'
printf 'output= %s/4_outs/14_vsearch_gg_picrust$(Process).out\n' "$ANALYSIS_PATH"
printf 'log  = %s/5_logs/14_vsearch_gg_picrust$(Process).log\n' "$ANALYSIS_PATH"
printf 'error= %s/6_errores/14_vsearch_gg_picrust$(Process).error\n' "$ANALYSIS_PATH"
echo 'request_cpus  = 5'

echo 'queue'
} > 14_picrust_clustering_condor.condor

15) Taxonomy blast.

In [ ]:
{
echo '#!/bin/bash'
echo 'export TMPDIR=/srv/home/anavarro/tmp'

echo 'qiime feature-classifier classify-consensus-blast \'
printf -- '--i-query %s/2_resultados/picrust/14_closedRef_forPICRUSt_13_8/clustered_sequences.qza \\\n' "$ANALYSIS_PATH"
printf -- '--i-reference-reads %s/7_databases/gg_13_8_otus97.qza \\\n' "$ANALYSIS_PATH"
printf -- '--i-reference-taxonomy %s/7_databases/gg_13_8_otus97_tax.qza \\\n' "$ANALYSIS_PATH"
echo '--p-perc-identity 0.97 \'
printf -- '--output-dir %s/2_resultados/picrust_analyses_13_8/15_picrust_taxonomy_blast_gg_13_8 \\\n' "$ANALYSIS_PATH"
echo '--verbose'

} > 15_taxonomy_blast_gg_13_8.sh

In [ ]:
{
printf 'executable 	= %s/3_ejecutables/picrust_analyses_excs/15_taxonomy_blast_gg_13_8.sh \n' "$ANALYSIS_PATH"
echo 'getenv		= True'

printf 'output		= %s/4_outs/15_tax_picrust_13_8$(Process).out\n' "$ANALYSIS_PATH"
printf 'log  		= %s/5_logs/15_tax_picrust_13_8$(Process).log\n' "$ANALYSIS_PATH"
printf 'error		= %s/6_errores/15_tax_picrust_13_8$(Process).error\n' "$ANALYSIS_PATH"

echo 'request_cpus	=0'

echo 'queue'
} > 15_taxonomy_blast_gg_13_8_submit.condor

In [ ]:
{
echo '#!/bin/bash'
printf 'export TMPDIR=%s\n' "$TMPDIR"

echo 'qiime feature-classifier classify-sklearn \'
printf -- '--i-reads %s/2_resultados/picrust/14_closedRef_forPICRUSt_13_8/clustered_sequences.qza \\\n' "$ANALYSIS_PATH"
printf -- '--i-classifier %s/7_databases/gg-13-8-99-nb-classifier.qza \\\n' "$ANALYSIS_PATH"
echo '--p-n-jobs -1 \'
echo '--p-confidence 0.8 \'
printf -- '--output-dir %s/2_resultados/picrust/15_picrust_taxonomy_output \\\n' "$ANALYSIS_PATH"
echo '--verbose'

} > 15_picrust_taxonomy_gg_sk.sh

In [ ]:
{
printf 'executable 	= %s/3_ejecutables/picrust_execs/15_picrust_taxonomy_gg_sk.sh\n' "$ANALYSIS_PATH" 
echo 'getenv		= True'
printf 'output		= %s/4_outs/15_picrust_taxonomy$(Process).out\n' "$ANALYSIS_PATH"
printf 'log  		= %s/5_logs/15_picrust_taxonomy$(Process).log\n' "$ANALYSIS_PATH"
printf 'error		= %s/6_errores/15_picrust_taxonomy$(Process).error\n' "$ANALYSIS_PATH"

echo 'request_cpus	= 5'
echo 'queue'
} > 15_picrust_taxonomy_gg_sk_submbit.condor

16) Exporting clustered output to biom file

In [ ]:
{
echo '#!/bin/bash'
printf 'export TMPDIR=%s\n' "$TMPDIR"

echo 'qiime tools export \'
printf -- '--input-path %s/2_resultados/picrust/14_closedRef_forPICRUSt_13_8/clustered_table.qza \\\n' "$ANALYSIS_PATH"
printf -- '--output-path %s/2_resultados/picrust/16_biom_13_8/' "$ANALYSIS_PATH"
} > 16_export_biom.sh

In [ ]:
{
printf 'executable 	= %s/3_ejecutables/picrust_execs/16_export_biom.sh\n' "$ANALYSIS_PATH"
echo 'getenv		= True'

printf 'output		= %s/4_outs/16_biom$(Process).out\n' "$ANALYSIS_PATH"
printf 'log  		= %s/5_logs/16_biom$(Process).log\n' "$ANALYSIS_PATH"
printf 'error		= %s/6_errores/16_biom$(Process).error\n' "$ANALYSIS_PATH"

echo 'request_cpus	= 5'

echo 'queue'
} > 16_export_biom.condor

In [ ]:
biom summarize-table -i ../../2_resultados/picrust/16_biom_13_8/feature-table.biom -o ../../2_resultados/picrust/16_biom_13_8/biom_sum.txt

17) Exporting taxonomy classificaction to TSV file

In [ ]:
{
echo '#!/bin/bash'
printf 'export TMPDIR=%s\n' "$TMPDIR"

echo 'qiime tools export \'
printf -- '--input-path %s/2_resultados/picrust/15_picrust_taxonomy_output/classification.qza \\\n' "$ANALYSIS_PATH"
printf -- '--output-path %s/2_resultados/picrust/17_picrust_tsv_gg_13_8' "$ANALYSIS_PATH"
} > 17_picrust_export_to_tsv_gg_13_8.sh

In [ ]:
{
printf 'executable 	= %s/3_ejecutables/17_picrust_export_to_tsv_gg_13_8.sh\n' "$ANALYSIS_PATH"
echo 'getenv		= True'
printf 'output		= %s/4_outs/17_export_tsv$(Process).out\n' "$ANALYSIS_PATH"
printf 'log  		= %s/5_logs/17_export_tsv$(Process).log\n' "$ANALYSIS_PATH"
printf 'error		= %s/6_errores/17_export_tsv$(Process).error\n' "$ANALYSIS_PATH"

echo 'request_cpus	= 5'

echo 'queue'
} > 17_picrust_export_to_tsv_gg_13_8_submit.condor

18) Joining biom with classification.

In [ ]:
{
echo '#!/bin/bash'
printf 'export TMPDIR=%s\n' "$TMPDIR"

echo 'biom add-metadata \'
printf -- '-i %s/2_resultados/picrust/16_biom_13_8/feature-table.biom  \\\n' "$ANALYSIS_PATH"
printf -- '-o %s/2_resultados/picrust/18_picrust_13_8_biom_tax.biom \\\n' "$ANALYSIS_PATH"
printf -- '--observation-metadata-fp %s/2_resultados/picrust/17_picrust_tsv_gg_13_8/4_taxonomy_updated_picrust.tsv \\\n' "$ANALYSIS_PATH"
echo '--observation-header OTUID,taxonomy \'
echo '--sc-separated taxonomy  \'
} > 18_join_biom_tax.sh

In [ ]:
biom summarize-table -i ../../2_resultados/picrust/18_picrust_13_8_biom_tax.biom -o ../../2_resultados/picrust/18_biom_sum.txt

19) Exporting biom as OTU table. Useful for other analyses.

In [ ]:
{
echo '#!/bin/bash'
printf 'export TMPDIR=%s\n' "$TMPDIR"

echo 'biom convert \'
printf -- '-i %s/2_resultados/picrust/18_picrust_13_8_biom_tax.biom \\\n' "$ANALYSIS_PATH"
printf -- '-o %s/2_resultados/picrust/19_table.from_biom_w_taxonomy.txt \\\n' "$ANALYSIS_PATH"
echo '--to-tsv \'
echo '--header-key taxonomy'
} > 19_biom_to_otu_table_gg_sk.sh

Install PICRUSt

In [ ]:
conda create --name picrust1.1.3

In [ ]:
source activate picrust1.1.3

In [ ]:
conda install -c bioconda picrust

In [ ]:
download_picrust_files.py

Step 1: Normalize OTU Table

In [ ]:
normalize_by_copy_number.py -i /srv/home/anavarro/agave_metagenom/2_resultados/picrust/18_picrust_13_8_biom_tax.biom -o /srv/home/anavarro/agave_metagenom/2_resultados/picrust/20_normalized_otus.biom

Step 2: Predict Functions For Metagenome

In [ ]:
predict_metagenomes.py -i 20_normalized_otus.biom -o 21_metagenome_predictions.biom -a 21_nsti_per_sample.tab

In [ ]:
predict_metagenomes.py -f -i 20_normalized_otus.tab -o 21_metagenome_predictions.tab -a 21_nsti_per_sample.tab

In [ ]:
{
echo '#!/bin/bash'
printf 'export TMPDIR=%s\n' "$TMPDIR"

echo 'predict_metagenomes.py \'
printf -- '-i %s/2_resultados/picrust/20_normalized_otus.biom \\\n' "$ANALYSIS_PATH"
printf -- '-o %s/2_resultados/picrust/21_metagenome_predictions.biom \\\n' "$ANALYSIS_PATH"
printf -- '-a %s/2_resultados/picrust/21_nsti_per_sample.tab \\\n' "$ANALYSIS_PATH"
echo '--verbose'
} > 21_predict_metagenome_biom.sh

In [ ]:
{
printf 'executable 	= %s/3_ejecutables/picrust_execs/21_predict_metagenome_biom.sh\n' "$ANALYSIS_PATH" 

echo 'getenv		= True'
printf 'output		= %s/4_outs/21_predict_metagenome$(Process).out\n' "$ANALYSIS_PATH" 
printf 'log  		= %s/5_logs/21_predict_metagenome$(Process).log\n' "$ANALYSIS_PATH" 
printf 'error		= %s/6_errores/21_predict_metagenome$(Process).error\n' "$ANALYSIS_PATH" 

echo 'request_cpus	=5'

echo 'queue'
} > 21_predict_metagenome_biom.condor

In [ ]:
condor_submit 21_predict_metagenome_biom.condor

In [ ]:
{
echo '#!/bin/bash'
printf 'export TMPDIR=%s\n' "$TMPDIR"

echo 'predict_metagenomes.py \'
printf -- '-i %s/2_resultados/picrust/20_normalized_otus.biom \\\n' "$ANALYSIS_PATH"
printf -- '-o %s/2_resultados/picrust/21_metagenome_predictions.tab \\\n' "$ANALYSIS_PATH"
printf -- '-a %s/2_resultados/picrust/21_nsti_per_sample_2.tab \\\n' "$ANALYSIS_PATH"
echo '-f'
} > 21_predict_metagenome_tab.sh

In [ ]:
{
printf 'executable 	= %s/3_ejecutables/picrust_execs/21_predict_metagenome_tab.sh\n' "$ANALYSIS_PATH" 

echo 'getenv		= True'
printf 'output		= %s/4_outs/21_predict_metagenome$(Process).out\n' "$ANALYSIS_PATH" 
printf 'log  		= %s/5_logs/21_predict_metagenome$(Process).log\n' "$ANALYSIS_PATH" 
printf 'error		= %s/6_errores/21_predict_metagenome$(Process).error\n' "$ANALYSIS_PATH" 

echo 'request_cpus	=5'

echo 'queue'
} > 21_predict_metagenome_tab.condor

In [ ]:
condor_submit 21_predict_metagenome_tab.condor

Bajar biom_to_stamp

In [ ]:
biom_to_stamp.py -m KEGG_Description 21_metagenome_predictions.biom > 21_metagenome_predictions.spf

Categorize by function. KO ids.

In [ ]:
categorize_by_function.py -i 21_metagenome_predictions.biom -c KEGG_Pathways -l 3 -o predicted_metagenomes.L3.tab

Metagenome contributions, for all functions (by KO ids).

In [ ]:
metagenome_contributions.py -i 20_normalized_otus.biom -t ko -o metagenome_contributions.all.tab

Higher classifications. KEGG Pathways. Only selected KO ids or functional categories.

In [ ]:
metagenome_contributions.py -i 20_normalized_otus.biom -f "Energy metabolism|Carbohydrate metabolism|Starch and sucrose metabolism|Bacterial chemotaxis|Amino acid metabolism|Metabolism of cofactors and vitamins" -t ko -o metagenome_contributions.selected_paths.tab

Energy metabolism

In [ ]:
metagenome_contributions.py -i 20_normalized_otus.biom -f "Energy metabolism" -t ko -o metagenome_contributions.energy_metabolism.tab

Carbohydrate metabolism

In [ ]:
metagenome_contributions.py -i 20_normalized_otus.biom -f "Carbohydrate metabolism" -t ko -o metagenome_contributions.carbohydrate_metabolism.tab

Starch and sucrose metabolism

In [ ]:
metagenome_contributions.py -i 20_normalized_otus.biom -f "Starch and sucrose metabolism" -t ko -o metagenome_contributions.starch_sucrose_metabolism.tab

Bacterial chemotaxis

In [ ]:
metagenome_contributions.py -i 20_normalized_otus.biom -f "Bacterial chemotaxis" -t ko -o metagenome_contributions.bacterial_chemotaxis.tab

Amino acid metabolism

In [ ]:
metagenome_contributions.py -i 20_normalized_otus.biom -f "Amino acid metabolism" -t ko -o metagenome_contributions.amino_acid_metabolism.tab

Metabolism of cofactors and vitamins

In [ ]:
metagenome_contributions.py -i 20_normalized_otus.biom -f "Metabolism of cofactors and vitamins" -t ko -o metagenome_contributions.cofactors_vitamins_metabolism.tab